In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from pyspark.sql import SparkSession

In [3]:
spark.sparkContext.setLogLevel("ERROR")


NameError: name 'spark' is not defined

In [4]:
spark = SparkSession.builder \
    .appName("KMeansexample") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

25/01/18 02:15:38 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.command

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/harshankurapati/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshankurapati/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/harshankurapati/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshankurapati/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protoco

In [ ]:
print(spark.version)


In [ ]:
data = [(1, "foo"), (2, "bar"), (3, "baz")]
df = spark.createDataFrame(data, ["id", "value"])
df.show()

# checking computational capacity

In [ ]:
import pandas as pd
import numpy as np
import time

# Generate a large datase
data = np.random.rand(1_100_000_000, 1)
df = pd.DataFrame(data, columns=["value"])

# Measure time
start_time = time.time()
sum_result = df["value"].sum()
end_time = time.time()

print("Without Spark:")
print(f"Sum: {sum_result}")
print(f"Time taken: {end_time - start_time:.2f} seconds")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum
import time

# Generate a large dataset
data = [(float(i),) for i in np.random.rand(1_100_000_000)]
spark_df = spark.createDataFrame(data, ["value"])

# Measure time
start_time = time.time()
sum_result = spark_df.select(spark_sum("value")).collect()[0][0]
end_time = time.time()

print("With Spark:")
print(f"Sum: {sum_result}")
print(f"Time taken: {end_time - start_time:.2f} seconds")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, sum as spark_sum
import time

# Create Spark session
spark = SparkSession.builder \
    .appName("Handle Large Dataset") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "32g") \
    .config("spark.sql.shuffle.partitions", "1000") \
    .getOrCreate()

# Generate a large dataset with random values
num_rows = 1_100_000_000
num_partitions = 1000
spark_df = spark.range(0, num_rows, numPartitions=num_partitions).select(rand().alias("value"))




In [ ]:
df = pd.read_csv("smoking_driking_dataset_Ver01.csv.gz")

In [ ]:
df

# Data Preparation
## Check for Missing Values

In [ ]:
df.isnull().sum()

## Convert Object Data types into Numerical Data types

In [ ]:
if "sex" in df.columns:
    one_hot = pd.get_dummies(df["sex"], prefix = 'Is_')
    df_ = pd.concat([df, one_hot], axis = 1)
    df = df_.drop(["sex"], axis = 1)

In [ ]:
df

In [ ]:
if "DRK_YN" in df.columns:    
    df = df.drop(["DRK_YN"], axis = 1)

    

## Standardising

In [ ]:
pipeline = Pipeline([
    ('standardiser', StandardScaler())
])

In [ ]:
Standard_df = pipeline.fit_transform(df)

In [ ]:
Standard_df

# Model

## Rough Model

In [ ]:
kmeans = KMeans(n_clusters =5,  n_init = 10, random_state = 42)
kmeans_pred = kmeans.fit_predict(Standard_df)
wcss = kmeans.inertia_

In [ ]:
centroids = kmeans.cluster_centers_

In [ ]:
plt.figure(figsize = (8,10))

plt.scatter(Standard_df[:,-1], Standard_df[:, -2], c = kmeans_pred, cmap = "viridis", alpha = 0.6)
plt.scatter(centroids[:,0], centroids[:, 1], c = "red", marker = 'X')

### The graph seems cluttered so Decided to try plot with the sample.

In [ ]:
np.random.seed(42)
sample_indices = np.random.choice(Standard_df.shape[0], replace = False, size = 1000)
sample_df = Standard_df[sample_indices]
sample_preds = kmeans_pred[sample_indices]

In [ ]:
plt.figure(figsize = (8,10))

plt.scatter(sample_df[:,2], sample_df[:, 8], c = sample_preds, cmap = "viridis", alpha = 0.6)
plt.scatter(centroids[:,0], centroids[:, 1], c = "red", marker = 'X')


In [ ]:
silhouette_score(Standard_df, kmeans_pred)